In [4]:
# taken partially from tutorial at: https://stmorse.github.io/journal/espn-fantasy-python.html
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests

league1 = 380134
league2 = 1553391
leagues=[league1, league2]

weeks = 15

scores={}

for i in range(len(leagues)):
    scores[i] = {}
for week in range(1, weeks+1):
    for league in range(len(leagues)):
        r = requests.get('http://games.espn.com/ffl/api/v2/scoreboard', 
                         params={'leagueId': leagues[league], 'seasonId': 2018, 'matchupPeriodId': week})
        scores[league][week] = r.json()

df = []
dfRecord = []
for league in range(len(leagues)):
    for week in scores[league]:
        temp = scores[league][week]['scoreboard']['matchups']
        for match in temp:
            
            df.append([week, 
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][0]['score'],
                       match['teams'][1]['score']
                      ])


df = pd.DataFrame(df, columns=['Week', 'HomeAbbrev', 'AwayAbbrev', 'HomeScore', 'AwayScore'])
df.head()
df = (df[['Week', 'HomeAbbrev', 'HomeScore']]
      .rename(columns={'HomeAbbrev': 'Abbrev', 'HomeScore': 'Score'})
      .append(df[['Week', 'AwayAbbrev', 'AwayScore']]
             .rename(columns={'AwayAbbrev': 'Abbrev', 'AwayScore': 'Score'}))
     )

# now adding the record:

for league in range(len(leagues)):
    temp2=scores[league][weeks]['scoreboard']['matchups']
    for match in temp2:
        for team in match['teams']:
#         append "points for" for debugging purposes
            dfRecord.append([
                            team['team']['teamAbbrev'],
                            team['team']['record']['pointsFor'],
                            team['team']['record']['pointsAgainst'],
                            team['team']['record']['overallWins'],
                            team['team']['record']['overallLosses'],
                            team['team']['record']['overallTies']
                            ])
dfRecord = pd.DataFrame(dfRecord, columns=["Abbrev", "Points For", "Points Against", "Overall Wins","Overall Losses", "Overall Ties"])
dfRecord = dfRecord.set_index('Abbrev')
df1 = df.groupby("Abbrev")["Score"].describe().drop("count", axis=1)
s1 = df.groupby("Abbrev")['Score'].sum()
finalDF = df1.join(dfRecord)
finalDF = finalDF.sort_values(by=['Overall Wins','Points For'], ascending=[False,False])
finalDF = finalDF.reset_index()
finalDF.index +=1
display(finalDF)

,Abbrev,mean,std,min,25%,50%,75%,max,Points For,Points Against,Overall Wins,Overall Losses,Overall Ties
1,PETR,130.934667,43.091544,58.32,102.04,116.48,173.36,193.48,1580.08,1068.68,11,2,0
2,LURI,130.794667,38.359263,93.26,104.36,122.84,130.02,220.64,1528.22,1212.44,10,3,0
3,AQP,117.993333,27.276421,92.68,101.62,109.20,124.97,204.50,1432.46,1137.16,9,4,0
4,AUTO,117.756000,39.636431,69.40,93.32,116.90,133.32,224.08,1377.40,1190.48,9,4,0
5,HARN,123.272000,50.318227,72.24,90.30,116.72,128.05,242.62,1374.94,1312.24,9,4,0
6,LEE,118.085333,41.800790,78.62,92.31,105.58,128.70,244.38,1363.88,1161.88,9,4,0
7,KANG,110.900000,37.926100,76.88,88.59,97.42,117.04,203.72,1268.26,1085.52,9,4,0
8,FITZ,117.070667,28.731279,79.00,100.46,110.00,127.70,176.94,1404.30,1229.16,8,5,0
9,DREW,103.045333,22.147346,61.36,87.49,106.30,122.36,141.16,1279.58,1197.88,8,5,0
10,QSTG,103.797333,34.918525,70.32,83.99,94.06,108.53,204.02,1247.78,1210.10,8,5,0
